1. Upload `.env` file into project directory `/home/studio-lab-user/sagemaker-studiolab-notebooks/chess-embedding-experiments/cloud/sagemaker`.
2. Execute `setup.sh` in the same directory.
3. Then you can run this notebook

In [7]:
%pip install -r /home/studio-lab-user/sagemaker-studiolab-notebooks/chess-embedding-experiments/requirements.txt

Ignoring pywin32: markers 'python_full_version >= "3.9.16" and python_version < "3.12" and sys_platform == "win32"' don't match your environment
Ignoring waitress: markers 'python_full_version >= "3.9.16" and python_version < "3.12" and platform_system == "Windows"' don't match your environment
     |████████████████████████████████| 351 kB 13.9 MB/s eta 0:00:01
     |████████████████████████████████| 443 kB 59.6 MB/s eta 0:00:01
     |████████████████████████████████| 138 kB 78.4 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 75.9 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 104.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 93.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 73.8 MB/s eta 0:00:01
     |████████████████████████████████| 18.2 MB 74.5 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 4.2 MB/s  eta 0:00:01
     |████████████████████████████████| 1.1 MB 58.2 MB/s eta 0:00:01
     |███████

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 588.3 MB 19 kB/s /s eta 0:00:01
     |████████████████████████████████| 226 kB 117.0 MB/s eta 0:00:01
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.6
    Uninstalling wcwidth-0.2.6:
      Successfully uninstalled wcwidth-0.2.6
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.5.0
    Uninstalling typing-extensions-4.5.0:
      Successfully uninstalled typing-extensions-4.5.0
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.2.0
    Uninstalling charset-normalizer-3.2.0:
      Successfully uninstalled charset-normalizer-3.2.0
  Attempting uninstall: cffi
    Found existing installation: cffi 1.15.1
    Uninstalling cffi-1.15.1:
      Successfully uninstalled cffi-1.15.1
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.

In [8]:
%cd /home/studio-lab-user/sagemaker-studiolab-notebooks/chess-embedding-experiments

/home/studio-lab-user/sagemaker-studiolab-notebooks/chess-embedding-experiments


In [ ]:
!python -m src.run.generate_positions

In [46]:
# if you want to remove checkpoints before starting the training
!rm -rf model/checkpoints

In [49]:
!python -m src.run.train

2023-10-01 21:52:16.185526: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-10-01 21:52:16.185595: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-10-01 21:52:16.185609: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023/10/01 21:52:17 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
Model: "autoe

Quick evaluation

In [39]:
path = !(echo $(pwd))

In [41]:
import sys
sys.path.insert(0, path[0])

import numpy as np
import chess
import tensorflow
from tensorflow import keras

from src.preprocessing.board_representation import tensor_to_board
from src.evaluation.visualisation import *
from src.utils.data_loader import load_train_test
from src.modeling.custom_losses import sum_squared_loss, pc_column_reg, pc_plane_reg, num_pc_reg, custom_regularized_loss

In [42]:
train_data, test_data = load_train_test("./data", "position_tensor_000")
print(len(train_data), len(test_data))

90000 10000


In [43]:
model = keras.models.load_model(
    "./model/20231001214209_autoencoder.keras",
    custom_objects={
        "sum_squared_loss": sum_squared_loss,
        "num_pc_reg": num_pc_reg,
        "pc_column_reg": pc_column_reg,
        "pc_plane_reg": pc_plane_reg,
        "custom_regularized_loss": custom_regularized_loss
    }
)

In [44]:
indices = [i for i in range(10)]
for index in indices:
    print(f"Compare boards for index {index}")
    sample = test_data[index].reshape((1,8,8,15))
    reconstruction = model.predict_on_batch(sample)
    print(model.evaluate(x=[reconstruction], y=[sample]))
    print(custom_regularized_loss(sample, reconstruction), sum_squared_loss(sample, reconstruction), num_pc_reg(sample, reconstruction), pc_column_reg(sample, reconstruction), pc_plane_reg(sample, reconstruction))
    binary_reconstruction = binarize_array(reconstruction)
    sample_board = tensor_to_board(sample[0])
    reconstructed_board = tensor_to_board(binary_reconstruction[0])
    print_board(sample_board)
    print("\n")
    print_board(reconstructed_board)

Compare boards for index 0
1/1 [==============================] - 1s 539ms/step - loss: 9.3125 - sum_squared_loss: 8.1875 - num_pc_reg: 0.1523 - pc_column_reg: 5.6562 - pc_plane_reg: 4.4688
[9.3125, 8.1875, 0.15234375, 5.65625, 4.46875]
tf.Tensor(8.75, shape=(), dtype=bfloat16) tf.Tensor(7.34375, shape=(), dtype=bfloat16) tf.Tensor(0.5625, shape=(), dtype=bfloat16) tf.Tensor(5.34375, shape=(), dtype=bfloat16) tf.Tensor(3.21875, shape=(), dtype=bfloat16)
. . . . . k . .
. p . . . p b .
p . . . p . p .
. . . p P . P .
. . . P . P . .
. N r q . . . .
P . . Q . . . .
. . . K . . . R




. . . . . . . .
. p . . . p b .
p . . . p . p .
. . . p P . P .
. . . P . P . .
. . . . . . . .
P . . . . . . .
. . . . . . . .


Compare boards for index 1
1/1 [==============================] - 0s 24ms/step - loss: 7.7188 - sum_squared_loss: 6.7188 - num_pc_reg: 0.1543 - pc_column_reg: 4.8438 - pc_plane_reg: 3.6094
[7.71875, 6.71875, 0.154296875, 4.84375, 3.609375]
tf.Tensor(7.59375, shape=(), dtype=bfloa

If that's promising then try to save artifacts

In [ ]:
!dvc add model

In [ ]:
!git add --all && git status

In [ ]:
!git commit -m "Colab model loss xy at batch yz"

In [ ]:
!git push origin master

In [ ]:
!dvc push -r colab_origin